In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

pipe = pipeline("text-generation", model="./LLaMA-Factory/models/tw_llama_step2", device_map="auto")

/home/ai2lab/anaconda3/envs/qwen2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 9/9 [00:05<00:00,  1.51it/s]


In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor

PATH = './LLaMA-Factory/models/tw_llama_step2'
tokenizer = AutoTokenizer.from_pretrained(PATH)
model = AutoModelForCausalLM.from_pretrained(PATH, device_map='auto')
processor = AutoProcessor.from_pretrained(PATH)

Loading checkpoint shards: 100%|██████████| 9/9 [00:05<00:00,  1.70it/s]


In [2]:
import pandas as pd
file_path = "./result.csv" # get the prediction from stage 1

step1_df = pd.read_csv(file_path)
step1_df['REF'] = step1_df['TARGET'].apply(lambda x: list(set([t.split('第')[0] for t in x.split(',')])))
step1_df

,id,TARGET,REF
0,test_0,"就業服務法第53條,就業服務法第54條",[就業服務法]
1,test_1,"民法第1055條,民法第1094條,民法第1089條,民法第1084條",[民法]
2,test_2,"全民健康保險法第84條,勞工保險條例第72條,勞工保險條例第11條,勞工保險條例第72條之1...","[勞工保險條例, 全民健康保險法]"
3,test_3,"民法第184條,民法第195條,民法第767條,民法第191條",[民法]
4,test_4,"中華民國刑法第305條,中華民國刑法第304條,中華民國刑法第306條",[中華民國刑法]
...,...,...,...
195,test_195,"土地徵收條例第28條,土地徵收條例第31條,土地徵收條例第10條,土地法第106條,土地法第...","[土地徵收條例, 土地法]"
196,test_196,道路交通管理處罰條例第92條,[道路交通管理處罰條例]
197,test_197,"行政訴訟法第3條,行政訴訟法第5條,行政訴訟法第8條,行政訴訟法第6條",[行政訴訟法]
198,test_198,勞動基準法第32條,[勞動基準法]


In [3]:
# format test data
test_data = {'id':[], 'situation':[]}
with open('./data/test_data.jsonl', 'r') as file:
    for line in file:
        test_data['id'].append(line.split('"id":"')[1].split('","title":')[0])
        title = line.split('","title":"')[1].split('","question":')[0]
        question = line.split('","question":"')[1].split('"}')[0]
        test_data['situation'].append(title+question if question!='NA' else title)

In [4]:
test_df = pd.DataFrame(test_data)
merge_df = pd.merge(step1_df, test_df, on='id')

In [5]:
merge_df.head()

,id,TARGET,REF,situation
0,test_0,"就業服務法第53條,就業服務法第54條",[就業服務法],"外籍勞工在合約到期後如何變更雇主?各位好,請教一下,如果外籍勞工在合約期滿後想要更換雇主,現..."
1,test_1,"民法第1055條,民法第1094條,民法第1089條,民法第1084條",[民法],"離婚後,我們雖然共同擁有監護權和扶養權,但前夫卻限制我與小孩進行通話、會面及同住。這樣的做法..."
2,test_2,"全民健康保險法第84條,勞工保險條例第72條,勞工保險條例第11條,勞工保險條例第72條之1...","[勞工保險條例, 全民健康保險法]","若未提交離職申請書,可能會引發哪些法律上的問題呢?因為只工作一天就離開,算不算試用期但最終選..."
3,test_3,"民法第184條,民法第195條,民法第767條,民法第191條",[民法],"自家樹葉自然而然掉落到樓下鄰居的地方,是否會涉及到任何法律上的違規行為?"
4,test_4,"中華民國刑法第305條,中華民國刑法第304條,中華民國刑法第306條",[中華民國刑法],"為了維護公共利益而奪取他人手機在電影院觀影時,如果我注意到前方有一位男子A在進行盜錄,出於保..."


In [6]:
# from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
# tokenizer = AutoTokenizer.from_pretrained("yentinglin/Llama-3-Taiwan-8B-Instruct")

print(tokenizer.vocab.get("否"), tokenizer.vocab.get("是"))

None None


In [76]:
import json
import torch
from tqdm import tqdm
# data source
with open('./data/law.json', 'r') as file:
    law_data = json.load(file)


total_aug_law = {'id':[], 'aug_law':[]}

for idx in range(len(merge_df)):
    this_aug_law = []
    scene = merge_df.iloc[idx]['situation']
    total_aug_law['id'].append(merge_df.iloc[idx]['id'])
    for e in merge_df.iloc[idx]['REF']:
        try:
            law = law_data[e]
            total_row = list(law.keys())
            num_law = len(law)
            print(f'start retrieval in: {e}, total length:{num_law}')
            for i in tqdm(range(2, num_law-3, 2)):
                key = total_row[i-2:i+3]
                sub_law = [e+k+':'+ law[k] for k in key]
                query = ','.join(sub_law)
                messages = [
                    {"role": "user", "content": f"參考台灣部份法律條文，判斷以下情境，是否有相關條文，與那條法律條文最相關: 法律條文:{query}\n情境：{scene}"},
                ]

                text = processor.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=True
                )
                inputs = processor(
                    text=[text],
                    padding=True,
                    return_tensors="pt",
                )
                inputs = inputs.to("cuda")
                generated_ids = model.generate(**inputs, max_new_tokens=2048, output_logits=True, return_dict_in_generate=True, do_sample=False, pad_token_id=tokenizer.eos_token_id)
                yesno_probs = torch.stack(generated_ids['logits'], dim=1).softmax(dim=-1)
                yesno_probs = yesno_probs[0, :1, [33476, 21043]].float().cpu().numpy()
                # print(generated_ids['sequences'])
                generated_ids_trimmed = [
                out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids['sequences'])
                ]

                # print(generated_ids_trimmed)
                output_text = processor.batch_decode(
                    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
                )
                # print(output_text)

                # print(yesno_probs[0][1])
                score = yesno_probs[0][1]
                T = 0.9 if e=='民法' or e=='中華民國刑法' else 0.8
                if score>T:
                    # print(score, output_text[0])
                    this_aug_law.append((score, output_text[0].split(',')[1].replace(' ', '')))
                # print(generated_ids.keys())
                # print(generated_ids['sequences'][:5])
                # print(generated_ids['logits'][0].shape)
        except:
            continue
            # break
    # print(this_aug_law)
    total_aug_law['aug_law'].append(this_aug_law)
    # break

start retrieval in: 就業服務法, total length:85


  0%|          | 0/40 [00:00<?, ?it/s]/home/ai2lab/anaconda3/envs/qwen2/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ai2lab/anaconda3/envs/qwen2/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 40/40 [00:24<00:00,  1.62it/s]


start retrieval in: 民法, total length:1439


100%|██████████| 717/717 [06:35<00:00,  1.81it/s]


start retrieval in: 勞工保險條例, total length:104


...

100%|██████████| 193/193 [02:01<00:00,  1.59it/s]


start retrieval in: 勞動基準法, total length:98


100%|██████████| 47/47 [00:35<00:00,  1.32it/s]


start retrieval in: 中華民國刑法, total length:415


100%|██████████| 205/205 [01:57<00:00,  1.75it/s]


In [60]:
print(total_aug_law)

{'id': ['test_0'], 'aug_law': [[(0.8366525, '就業服務法第5條'), (0.82215357, '就業服務法第7條'), (0.8982708, '就業服務法第55條'), (0.8449066, '就業服務法第70條')]]}


In [77]:
aug_df = pd.DataFrame(total_aug_law)
aug_merge_df = pd.merge(merge_df, aug_df, on='id')

In [78]:
aug_merge_df.head()

,id,TARGET,REF,situation,aug_law
0,test_0,"就業服務法第53條,就業服務法第54條",[就業服務法],"外籍勞工在合約到期後如何變更雇主?各位好,請教一下,如果外籍勞工在合約期滿後想要更換雇主,現...","[(0.82215357, 就業服務法第7條)]"
1,test_1,"民法第1055條,民法第1094條,民法第1089條,民法第1084條",[民法],"離婚後,我們雖然共同擁有監護權和扶養權,但前夫卻限制我與小孩進行通話、會面及同住。這樣的做法...","[(0.9684313, 民法第128條), (0.932414, 民法第136條), (0..."
2,test_2,"全民健康保險法第84條,勞工保險條例第72條,勞工保險條例第11條,勞工保險條例第72條之1...","[勞工保險條例, 全民健康保險法]","若未提交離職申請書,可能會引發哪些法律上的問題呢?因為只工作一天就離開,算不算試用期但最終選...","[(0.86816466, 勞工保險條例第3條), (0.9108135, 勞工保險條例第1..."
3,test_3,"民法第184條,民法第195條,民法第767條,民法第191條",[民法],"自家樹葉自然而然掉落到樓下鄰居的地方,是否會涉及到任何法律上的違規行為?","[(0.9707046, 民法第185條), (0.9825011, 民法第188條), (..."
4,test_4,"中華民國刑法第305條,中華民國刑法第304條,中華民國刑法第306條",[中華民國刑法],"為了維護公共利益而奪取他人手機在電影院觀影時,如果我注意到前方有一位男子A在進行盜錄,出於保...","[(0.94947684, 中華民國刑法第14條), (0.9950848, 中華民國刑法第..."


In [79]:
aug_merge_df.to_csv('./out_all.csv', index=False)  

In [90]:
top_k = 3
out_dict = {'id':[], 'TARGET':[]}
for i in range(len(aug_merge_df)):
   
    r = aug_merge_df.iloc[i]
    out_dict['id'].append(r['id'])
    t = r['TARGET']
    ref = r['aug_law']

    ref.sort(key = lambda x: x[0], reverse=True)
    # print(ref)
    select_law = ','.join([element[1] for element in ref[:top_k]])
    out_dict['TARGET'].append(f'{t},{select_law}')


print(out_dict)

{'id': ['test_0', 'test_1', 'test_2', 'test_3', 'test_4', 'test_5', 'test_6', 'test_7', 'test_8', 'test_9', 'test_10', 'test_11', 'test_12', 'test_13', 'test_14', 'test_15', 'test_16', 'test_17', 'test_18', 'test_19', 'test_20', 'test_21', 'test_22', 'test_23', 'test_24', 'test_25', 'test_26', 'test_27', 'test_28', 'test_29', 'test_30', 'test_31', 'test_32', 'test_33', 'test_34', 'test_35', 'test_36', 'test_37', 'test_38', 'test_39', 'test_40', 'test_41', 'test_42', 'test_43', 'test_44', 'test_45', 'test_46', 'test_47', 'test_48', 'test_49', 'test_50', 'test_51', 'test_52', 'test_53', 'test_54', 'test_55', 'test_56', 'test_57', 'test_58', 'test_59', 'test_60', 'test_61', 'test_62', 'test_63', 'test_64', 'test_65', 'test_66', 'test_67', 'test_68', 'test_69', 'test_70', 'test_71', 'test_72', 'test_73', 'test_74', 'test_75', 'test_76', 'test_77', 'test_78', 'test_79', 'test_80', 'test_81', 'test_82', 'test_83', 'test_84', 'test_85', 'test_86', 'test_87', 'test_88', 'test_89', 'test_90', '

In [91]:
out_df = pd.DataFrame(out_dict)
out_df.to_csv('./result_k=3.csv', index=False)  